In [1]:
from bs4 import BeautifulSoup
import requests
import re

def getMovieLinks():
    response = requests.get("http://www.movie-list.com/archive.php")
    bs0bj = BeautifulSoup(response.text,'lxml')
    baseUrl = 'http://www.movie-list.com'
    links = []
    for link in bs0bj.findAll('a',href=re.compile('/trailers/.+')):
        newUrl = baseUrl+link.attrs['href']
        if newUrl not in links:
            links.append(newUrl)
    return links

In [2]:
#找出演员的函数
def F_Actor(Mdiv):
    Actors = []
    try:
        for x in Mdiv.find('span',class_='cast').findAll('span',itemprop='name'):
            Actors.append(x.text.lstrip())
    except AttributeError as err:#对于没有演员的电影，将演员项置为空
        Actors.append(None)
    return Actors

In [3]:
def F_IMDb(Mbs0bj):
    linkpattern = re.compile('imdb.com/title/.*')
    link = Mbs0bj.find('a',href=linkpattern)
    #print(link['href'])
    if(link):
        link = link['href']
        #四种模式的id
        #前两种比较常见，后两种只有两个例子 可以作为补充
        pattern1 = re.compile(r'title/(tt\d*)/')
        pattern2 = re.compile(r'title/(tt\d*)')
        pattern3 = re.compile(r'title/(\w*)/')
        pattern4 = re.compile(r'http://(w*)\.(\w*)(.com)/')
        try:
            id = re.search(pattern1,link).group()
            #id = re.search(pattern3,link).group()  
        except:
            id = re.search(pattern2,link).group()
            #id = re.search(pattern4,link).group()
        #IMDb = 'http://www.imdb.com/'+id
        IMDb = id
     #提取id号重新组合为新的imdb
    else:
        IMDb = None
    return IMDb

In [4]:
#总的爬虫函数
def crawlMovieItem(url):
    headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"
}
    
    Mresponse = requests.get(url,headers=headers)
    Mbs0bj    = BeautifulSoup(Mresponse.text,'lxml')
    #需要抓取的内容主要在以下位置
    Mdiv = Mbs0bj.find('div',style='padding-left:168px;')
    Name = Mdiv.h1.text.lstrip()
    Releasedate = Mdiv.find('span',style="color:#181818; font-size:12px;").text
    Director = Mdiv.find('span',itemprop="director").find('span',itemprop='name').text
    Genre  = []
    for x in Mdiv.findAll('span',itemprop="genre"):
        Genre.append(x.text.lstrip())
    Actors = F_Actor(Mdiv)
    #IMDb网络电影资料库
    IMDb = F_IMDb(Mbs0bj)
    RatingCount = Mbs0bj.find('span',itemprop='ratingCount').text.lstrip()
    RatingValue = Mbs0bj.find('span',itemprop='ratingValue').text.lstrip()
    Movie = {'Name':Name,'ReleaseDate':Releasedate,'Director':Director,'Actors':Actors,
             'Genre':Genre,'IMDb':IMDb,'RatingCount':RatingCount,'RatingValue':RatingValue}
    return Movie

In [5]:
import json
def crawl():
    out = open('H:desktop/Movie.txt','w')
    number = 0
    try:
        links = getMovieLinks()
        print("the number of movie is",len(links))
        for link in links:
            number += 1
            try:
                movie = crawlMovieItem(link)
                Jmovie = json.dumps(movie)
                print(Jmovie,file=out)
            except AttributeError as err:
                print(number,file=out)
                print(err,number)
            except ConnectionError:
                print(number,file=out)
                print('ConnectionError',file=out)
    except ConnectionError:
        print('ConnectionError')
        #出现连接错误提前关闭文件
        out.close()
    out.close()
crawl()

the number of movie is 5942
'NoneType' object has no attribute 'h1' 1738
'NoneType' object has no attribute 'h1' 1739
'NoneType' object has no attribute 'h1' 1740
'NoneType' object has no attribute 'h1' 1741
'NoneType' object has no attribute 'h1' 1742
'NoneType' object has no attribute 'h1' 1743
'NoneType' object has no attribute 'group' 3475
'NoneType' object has no attribute 'group' 5350


In [12]:
Movie1 = crawlMovieItem(links[3450])
#a代表追加写入，不覆盖原文件
out = open('H:desktop/Movie.txt','a')
print(Movie1,file=out)
out.close()

NameError: name 'links' is not defined

In [13]:
Movie2 = crawlMovieItem(links[5310])
out = open('H:desktop/Movie.txt','a')
print(Movie2,file=out)
out.close()

NameError: name 'links' is not defined

# 将字典转为json

In [6]:
import json
d = {'s':'you','d':'are'}   #给一个字典
j = json.dumps(d)
print(type(j))

<class 'str'>
